# Regresión de Precios de Vivienda en California con Redes Neuronales (PyTorch) y Regularización

**Disciplina:** Aprendizaje Profundo, Redes Neuronales, Regresión, PyTorch

**Objetivo:**
El objetivo de este notebook es construir, entrenar y evaluar una red neuronal para predecir los precios medianos de las viviendas en California utilizando PyTorch. Se incorporarán técnicas de preprocesamiento, regularización (Weight Decay y Dropout) y manejo explícito del bucle de entrenamiento con funcionalidades equivalentes a callbacks (EarlyStopping, ModelCheckpoint, ReduceLROnPlateau) para mejorar el entrenamiento y la robustez del modelo de regresión.

## 1. Carga de Librerías y Configuración Inicial

**Propósito de esta sección:**
Importar todas las bibliotecas necesarias y configurar el entorno para el análisis.

**Bibliotecas Clave:**
* **`numpy`, `pandas`**: Para manipulación de datos.
* **`matplotlib.pyplot`, `seaborn`**: Para visualizaciones.
* **`sklearn.datasets`**: Para cargar el dataset California Housing.
* **`sklearn.model_selection`**: Para `train_test_split`.
* **`sklearn.preprocessing`**: Para `StandardScaler`.
* **`sklearn.metrics`**: Para `mean_squared_error`, `mean_absolute_error`, `r2_score`.
* **`torch`, `torch.nn`, `torch.optim`, `torch.utils.data`**: Para PyTorch.

In [ ]:
# Comandos mágicos de IPython (opcional en scripts)
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# Importación de bibliotecas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import copy

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Importar PyTorch
PYTORCH_IMPORTED_SUCCESSFULLY = False # <--- DEFINIR LA BANDERA AQUÍ
try:
    import torch
    import torch.nn as nn
    import torch.optim as optim
    from torch.utils.data import TensorDataset, DataLoader
    print(f"Biblioteca 'torch' importada correctamente. Versión: {torch.__version__}")
    PYTORCH_IMPORTED_SUCCESSFULLY = True # <--- ACTUALIZAR LA BANDERA
except ImportError as e:
    print(f"Error al importar 'torch': {e}")
    print("Por favor, instálala con 'pip install torch torchvision torchaudio' (o según tu sistema).")
    print("El script continuará, pero las secciones de PyTorch probablemente fallarán.")


# Configuración para reproducibilidad
SEED = 42
np.random.seed(SEED)
if PYTORCH_IMPORTED_SUCCESSFULLY:
    torch.manual_seed(SEED)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(SEED)

# Configuración de dispositivo
device = torch.device("cuda" if PYTORCH_IMPORTED_SUCCESSFULLY and torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")

# Configuración de estilo y visualización
sns.set(style="whitegrid")
plt.rcParams['figure.figsize'] = [10, 6]
plt.rcParams['figure.dpi'] = 100
plt.rcParams['font.size'] = 10

## 2. Funciones Personalizadas

### Descripción de la Función: `cargar_y_preparar_datos_california_pytorch`

**Objetivo Principal:**
Cargar el dataset California Housing, preprocesarlo (escalado), convertir a tensores PyTorch y crear DataLoaders.

**Características:**
* **Procesamiento:**
    1. Carga el dataset.
    2. Escala características X (y opcionalmente y) con `StandardScaler`.
    3. Convierte a tensores PyTorch (y debe ser `(N,1)` para regresión con `MSELoss`).
    4. Divide en entrenamiento, validación y prueba.
    5. Crea `DataLoader` para cada conjunto.
* **Valor de Retorno:** DataLoaders, scalers, nombres, datos NumPy de entrenamiento, validación y prueba.

In [ ]:
def cargar_y_preparar_datos_california_pytorch(batch_size=32, test_size=0.2, val_size=0.1, random_state=SEED, scale_target=False):
    print("Cargando y preparando el dataset California Housing para PyTorch...")
    housing = fetch_california_housing()
    X_np = housing.data
    y_np = housing.target
    feature_names = housing.feature_names

    scaler_X = StandardScaler()
    X_scaled_np = scaler_X.fit_transform(X_np)
    
    y_processed_np = y_np # y original para las métricas finales si no se escala y
    scaler_y = None
    if scale_target:
        scaler_y = StandardScaler()
        # y_processed_np es la versión que irá a los tensores y al entrenamiento
        y_processed_np = scaler_y.fit_transform(y_np.reshape(-1, 1)).flatten()

    X_temp_np, X_test_np, y_temp_np, y_test_np = train_test_split(
        X_scaled_np, y_processed_np, test_size=test_size, random_state=random_state
    )
    
    val_proportion_of_temp = val_size / (1 - test_size) if (1-test_size) > 0 else 0
    if val_proportion_of_temp == 0 and len(X_temp_np) > 0:
        X_train_np, X_val_np, y_train_np, y_val_np = X_temp_np, np.array([]), y_temp_np, np.array([])
        X_val_np = np.empty((0, X_train_np.shape[1]), dtype=X_train_np.dtype)
        y_val_np = np.empty((0,), dtype=y_train_np.dtype)
    elif val_proportion_of_temp > 0:
         X_train_np, X_val_np, y_train_np, y_val_np = train_test_split(
            X_temp_np, y_temp_np, test_size=val_proportion_of_temp, random_state=random_state
        )
    else: # X_temp_np is empty
        X_train_np, X_val_np, y_train_np, y_val_np = X_temp_np, X_temp_np, y_temp_np, y_temp_np


    train_loader, val_loader, test_loader = None, None, None
    if PYTORCH_IMPORTED_SUCCESSFULLY:
        X_train_torch = torch.tensor(X_train_np, dtype=torch.float32)
        y_train_torch = torch.tensor(y_train_np, dtype=torch.float32).unsqueeze(1)
        X_val_torch = torch.tensor(X_val_np, dtype=torch.float32)
        y_val_torch = torch.tensor(y_val_np, dtype=torch.float32).unsqueeze(1)
        X_test_torch = torch.tensor(X_test_np, dtype=torch.float32)
        y_test_torch = torch.tensor(y_test_np, dtype=torch.float32).unsqueeze(1)
        
        train_dataset = TensorDataset(X_train_torch, y_train_torch)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        
        if X_val_torch.shape[0] > 0:
            val_dataset = TensorDataset(X_val_torch, y_val_torch)
            val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
        
        test_dataset = TensorDataset(X_test_torch, y_test_torch)
        test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    print(f"\nDimensiones NumPy: X_train: {X_train_np.shape}, y_train: {y_train_np.shape}")
    print(f"Dimensiones NumPy: X_val: {X_val_np.shape}, y_val: {y_val_np.shape}")
    print(f"Dimensiones NumPy: X_test: {X_test_np.shape}, y_test: {y_test_np.shape}")
    if train_loader:
        print(f"Tamaños DataLoaders: Train: {len(train_loader.dataset)}, Val: {len(val_loader.dataset) if val_loader else 0}, Test: {len(test_loader.dataset)}")
    
    # Devolver y_test_np (que está escalado o no según scale_target) para la evaluación final
    # y y_np (el original, sin escalar) para referencia si es necesario.
    return (train_loader, val_loader, test_loader, 
            scaler_X, scaler_y, feature_names, 
            X_train_np, X_val_np, X_test_np,
            y_train_np, y_val_np, y_test_np, y_np) # Añadir y_np original

### Descripción de la Clase: `HousingRegressorNet`
(Sin cambios)

In [ ]:
if PYTORCH_IMPORTED_SUCCESSFULLY:
    class HousingRegressorNet(nn.Module):
        def __init__(self, input_dim, dropout_rate=0.3):
            super(HousingRegressorNet, self).__init__()
            # Aumentar un poco la complejidad para California Housing
            self.fc1 = nn.Linear(input_dim, 128) 
            self.relu1 = nn.ReLU()
            # self.bn1 = nn.BatchNorm1d(128) # Opcional Batch Norm
            self.dropout1 = nn.Dropout(dropout_rate)
            
            self.fc2 = nn.Linear(128, 64)
            self.relu2 = nn.ReLU()
            # self.bn2 = nn.BatchNorm1d(64)
            self.dropout2 = nn.Dropout(dropout_rate)
            
            self.fc3 = nn.Linear(64, 32)
            self.relu3 = nn.ReLU()
            
            self.fc4 = nn.Linear(32, 1)

        def forward(self, x):
            x = self.fc1(x)
            x = self.relu1(x)
            # x = self.bn1(x)
            x = self.dropout1(x)
            
            x = self.fc2(x)
            x = self.relu2(x)
            # x = self.bn2(x)
            x = self.dropout2(x)
            
            x = self.fc3(x)
            x = self.relu3(x)
            
            x = self.fc4(x)
            return x
else:
    HousingRegressorNet = None
    print("PyTorch no importado, HousingRegressorNet no definido.")

### Descripción de la Función: `entrenar_modelo_regresion_pytorch`
(Sin cambios, pero su ejecución dependerá de PYTORCH_IMPORTED_SUCCESSFULLY)

In [ ]:
def entrenar_modelo_regresion_pytorch(model, train_loader, val_loader, criterion, optimizer, scheduler, 
                                      num_epochs, patience, model_save_path, device):
    if not PYTORCH_IMPORTED_SUCCESSFULLY or model is None:
        print("PyTorch no disponible o modelo no definido. Saltando entrenamiento de regresión.")
        return {"train_loss": [], "val_loss": [], "train_mae": [], "val_mae": []}

    print(f"\nIniciando entrenamiento de regresión en {device} por {num_epochs} épocas...")
    
    train_losses, val_losses = [], []
    train_maes, val_maes = [], [] 
    
    best_val_loss = float('inf')
    epochs_no_improve = 0
    best_model_state = None

    for epoch in range(num_epochs):
        model.train()
        running_loss_train = 0.0
        running_mae_train = 0.0
        
        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            running_loss_train += loss.item() * inputs.size(0)
            running_mae_train += torch.abs(outputs - targets).sum().item()
            
        epoch_loss_train = running_loss_train / len(train_loader.dataset) if len(train_loader.dataset) > 0 else 0
        epoch_mae_train = running_mae_train / len(train_loader.dataset) if len(train_loader.dataset) > 0 else 0
        train_losses.append(epoch_loss_train)
        train_maes.append(epoch_mae_train)

        epoch_loss_val = float('nan')
        epoch_mae_val = float('nan')

        if val_loader and len(val_loader.dataset) > 0:
            model.eval()
            running_loss_val = 0.0
            running_mae_val = 0.0
            with torch.no_grad():
                for inputs, targets in val_loader:
                    inputs, targets = inputs.to(device), targets.to(device)
                    outputs = model(inputs)
                    loss_val_batch = criterion(outputs, targets)
                    running_loss_val += loss_val_batch.item() * inputs.size(0)
                    running_mae_val += torch.abs(outputs - targets).sum().item()
            
            epoch_loss_val = running_loss_val / len(val_loader.dataset) if len(val_loader.dataset) > 0 else 0
            epoch_mae_val = running_mae_val / len(val_loader.dataset) if len(val_loader.dataset) > 0 else 0
        
        val_losses.append(epoch_loss_val)
        val_maes.append(epoch_mae_val)
        
        print(f"Época {epoch+1}/{num_epochs} | "
              f"Pérdida Ent. (MSE): {epoch_loss_train:.4f} | MAE Ent.: {epoch_mae_train:.4f} | "
              f"Pérdida Val. (MSE): {epoch_loss_val:.4f} | MAE Val.: {epoch_mae_val:.4f}")

        if scheduler and not np.isnan(epoch_loss_val):
            scheduler.step(epoch_loss_val)

        if not np.isnan(epoch_loss_val) and epoch_loss_val < best_val_loss:
            best_val_loss = epoch_loss_val
            best_model_state = copy.deepcopy(model.state_dict())
            torch.save(best_model_state, model_save_path)
            print(f"  Mejora en validación (Pérdida Val.: {best_val_loss:.4f}). Guardando modelo en {model_save_path}")
            epochs_no_improve = 0
        elif not np.isnan(epoch_loss_val) :
            epochs_no_improve += 1
            print(f"  Sin mejora en validación por {epochs_no_improve} épocas.")
            if epochs_no_improve >= patience:
                print(f"Early stopping activado. Mejor val_loss: {best_val_loss:.4f}")
                break
        elif np.isnan(epoch_loss_val) and epoch == 0:
             best_model_state = copy.deepcopy(model.state_dict())
             torch.save(best_model_state, model_save_path)
             print(f"  No hay set de validación. Guardando modelo de época 1 en {model_save_path}")
                
    if best_model_state:
        print(f"Cargando los pesos del mejor modelo desde {model_save_path} (mejor val_loss: {best_val_loss:.4f})")
        model.load_state_dict(best_model_state)
        
    return {"train_loss": train_losses, "val_loss": val_losses, 
            "train_mae": train_maes, "val_mae": val_maes}

### Descripción de la Función: `graficar_historial_regresion_pytorch`
(Sin cambios)

In [ ]:
def graficar_historial_regresion_pytorch(history_dict):
    print("\nGraficando historial de entrenamiento (Regresión PyTorch)...")
    train_loss = history_dict.get('train_loss')
    val_loss = history_dict.get('val_loss')
    train_mae = history_dict.get('train_mae')
    val_mae = history_dict.get('val_mae')

    val_loss_plot = [v for v in val_loss if not np.isnan(v)] if val_loss else []
    val_mae_plot = [v for v in val_mae if not np.isnan(v)] if val_mae else []
    
    epochs_range_train = range(len(train_loss if train_loss else train_mae))
    epochs_range_val = range(len(val_loss_plot if val_loss_plot else val_mae_plot))


    plt.figure(figsize=(14, 5))

    plt.subplot(1, 2, 1)
    if train_loss:
        plt.plot(epochs_range_train, train_loss, label='Pérdida (MSE) - Entrenamiento')
        if val_loss_plot:
            plt.plot(epochs_range_val, val_loss_plot, label='Pérdida (MSE) - Validación')
        plt.title('Pérdida (MSE) de Entrenamiento y Validación')
        plt.xlabel('Épocas'); plt.ylabel('MSE'); plt.legend()

    plt.subplot(1, 2, 2)
    if train_mae:
        plt.plot(epochs_range_train, train_mae, label='MAE - Entrenamiento')
        if val_mae_plot:
            plt.plot(epochs_range_val, val_mae_plot, label='MAE - Validación')
        plt.title('Error Absoluto Medio (MAE)'); plt.xlabel('Épocas'); plt.ylabel('MAE')
        plt.legend()
    plt.show()

### Descripción de la Función: `evaluar_modelo_regresion_pytorch`
(Sin cambios)

In [ ]:
def evaluar_modelo_regresion_pytorch(model, test_loader, criterion, 
                                     X_test_numpy_for_eval, y_test_numpy_for_eval, 
                                     scaler_y=None, device=device):
    if not PYTORCH_IMPORTED_SUCCESSFULLY or model is None:
        print("PyTorch no disponible o modelo no definido. Saltando evaluación de regresión.")
        return

    print("\nEvaluando el modelo de regresión PyTorch en el conjunto de prueba...")
    model.eval()
    test_loss_mse_sum = 0.0
    test_mae_sum = 0.0 # MAE calculado sobre los datos (posiblemente escalados) del test_loader
    all_preds_list = []
    
    with torch.no_grad():
        for inputs, targets in test_loader: 
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets) 
            test_loss_mse_sum += loss.item() * inputs.size(0)
            test_mae_sum += torch.abs(outputs - targets).sum().item()
            all_preds_list.append(outputs.cpu().numpy()) 
            
    avg_test_loss_mse = test_loss_mse_sum / len(y_test_numpy_for_eval) if len(y_test_numpy_for_eval) > 0 else 0
    avg_test_mae_pytorch = test_mae_sum / len(y_test_numpy_for_eval) if len(y_test_numpy_for_eval) > 0 else 0
    
    print(f"Pérdida Promedio (MSE) en el conjunto de prueba: {avg_test_loss_mse:.4f}")
    print(f"MAE Promedio (calculado por PyTorch sobre datos de test_loader): {avg_test_mae_pytorch:.4f}")

    y_pred_from_loader = np.concatenate(all_preds_list).flatten()
    
    # y_test_numpy_for_eval es el y_test que corresponde a X_test_numpy_for_eval
    # y_pred_from_loader son las predicciones para X_test_numpy_for_eval
    # Ambos están en la misma escala (la que se usó para el entrenamiento/test_loader)
    
    y_test_final_for_metrics = y_test_numpy_for_eval 
    y_pred_final_for_metrics = y_pred_from_loader

    y_test_to_plot = y_test_numpy_for_eval
    y_pred_to_plot = y_pred_from_loader
    
    if scaler_y: # Si el target original (y_test_numpy_for_eval) fue escalado
        print("Desescalando predicciones y valores reales para R2 e interpretación...")
        y_test_to_plot = scaler_y.inverse_transform(y_test_numpy_for_eval.reshape(-1,1)).flatten()
        y_pred_to_plot = scaler_y.inverse_transform(y_pred_from_loader.reshape(-1,1)).flatten()
        
        mae_descalado = mean_absolute_error(y_test_to_plot, y_pred_to_plot)
        print(f"MAE (desescalado, unidades originales) en el conjunto de prueba: {mae_descalado:.4f}")

    r2 = r2_score(y_test_to_plot, y_pred_to_plot)
    print(f"R^2 Score: {r2:.4f}")

    plt.figure(figsize=(8, 8))
    plt.scatter(y_test_to_plot, y_pred_to_plot, alpha=0.5)
    min_val_plot = min(np.min(y_test_to_plot), np.min(y_pred_to_plot))
    max_val_plot = max(np.max(y_test_to_plot), np.max(y_pred_to_plot))
    plt.plot([min_val_plot, max_val_plot], [min_val_plot, max_val_plot], '--', color='red', lw=2)
    plt.xlabel('Valores Reales (Unidad Original si desescalado)')
    plt.ylabel('Predicciones (Unidad Original si desescalado)')
    plt.title('Predicciones vs. Valores Reales (PyTorch)')
    plt.grid(True); plt.show()

    residuals = y_test_to_plot - y_pred_to_plot
    plt.figure(figsize=(8, 6))
    sns.histplot(residuals, kde=True)
    plt.xlabel('Residuales (Real - Predicción)'); plt.ylabel('Frecuencia')
    plt.title('Distribución de los Residuales (PyTorch)'); plt.axvline(0, color='red', linestyle='--')
    plt.grid(True); plt.show()

    plt.figure(figsize=(8, 6))
    plt.scatter(y_pred_to_plot, residuals, alpha=0.5)
    plt.xlabel('Valores Predichos'); plt.ylabel('Residuales')
    plt.title('Residuales vs. Valores Predichos (PyTorch)'); plt.axhline(0, color='red', linestyle='--')
    plt.grid(True); plt.show()

## 3. Desarrollo del Ejercicio: Regresión de Precios de Vivienda con PyTorch

### 3.1. Carga y Preparación de Datos

In [ ]:
BATCH_SIZE_HOUSING = 64
(train_loader_housing, val_loader_housing, test_loader_housing, 
 scaler_X_housing, scaler_y_housing, housing_feature_names,
 X_train_housing_np, X_val_housing_np, X_test_housing_np,
 y_train_housing_np, y_val_housing_np, y_test_housing_np,
 y_housing_original_np # y original sin escalar
 ) = cargar_y_preparar_datos_california_pytorch(
        batch_size=BATCH_SIZE_HOUSING, 
        scale_target=True # Probar escalar el target para regresión
    )

### 3.2. Creación del Modelo de Regresión

In [ ]:
modelo_housing_pytorch = None
if PYTORCH_IMPORTED_SUCCESSFULLY and HousingRegressorNet is not None:
    if X_train_housing_np.shape[0] > 0:
        input_dim_housing_actual = X_train_housing_np.shape[1]
    elif train_loader_housing and len(train_loader_housing.dataset) > 0:
        sample_inputs_housing, _ = next(iter(train_loader_housing))
        input_dim_housing_actual = sample_inputs_housing.shape[1]
    else:
        print("No se pudo determinar input_dim para el modelo Housing. Usando valor por defecto 8.")
        input_dim_housing_actual = 8 # California Housing tiene 8 características

    modelo_housing_pytorch = HousingRegressorNet(input_dim_housing_actual, dropout_rate=0.2).to(device)
    print("\nResumen del modelo PyTorch de Regresión (estructura):")
    print(modelo_housing_pytorch)
else:
    print("PyTorch no se importó o HousingRegressorNet no definido, saltando creación de modelo de regresión.")

### 3.3. Definición de Pérdida, Optimizador y Scheduler

In [ ]:
criterion_housing, optimizer_housing, scheduler_housing = None, None, None
if PYTORCH_IMPORTED_SUCCESSFULLY and modelo_housing_pytorch:
    criterion_housing = nn.MSELoss() 
    optimizer_housing = optim.Adam(modelo_housing_pytorch.parameters(), lr=0.001, weight_decay=0.0001) 
    scheduler_housing = optim.lr_scheduler.ReduceLROnPlateau(optimizer_housing, 'min', patience=10, factor=0.5, verbose=True, min_lr=1e-6)
else:
    print("Modelo de regresión no definido.")

### 3.4. Entrenamiento del Modelo

In [ ]:
NUM_EPOCHS_HOUSING = 200 # Reducido para ejecución más rápida
PATIENCE_HOUSING = 20 
MODEL_SAVE_PATH_HOUSING = 'best_housing_model_pytorch.pth'
history_housing_pytorch = None

if modelo_housing_pytorch and criterion_housing and optimizer_housing and train_loader_housing and val_loader_housing:
    history_housing_pytorch = entrenar_modelo_regresion_pytorch(
        modelo_housing_pytorch, train_loader_housing, val_loader_housing,
        criterion_housing, optimizer_housing, scheduler_housing,
        num_epochs=NUM_EPOCHS_HOUSING,
        patience=PATIENCE_HOUSING,
        model_save_path=MODEL_SAVE_PATH_HOUSING,
        device=device
    )
else:
    print("Componentes de entrenamiento de regresión no disponibles. Saltando entrenamiento.")

### 3.5. Visualización del Historial de Entrenamiento

In [ ]:
if history_housing_pytorch:
    graficar_historial_regresion_pytorch(history_housing_pytorch)

### 3.6. Evaluación del Modelo

In [ ]:
if modelo_housing_pytorch and history_housing_pytorch and criterion_housing and test_loader_housing:
    # y_test_housing_np fue devuelto por la función de carga y ya está en la escala correcta
    # (escalado si scale_target=True, no escalado si scale_target=False)
    # para comparar con las predicciones del modelo que también estarán en esa escala.
    evaluar_modelo_regresion_pytorch(
        modelo_housing_pytorch, test_loader_housing, criterion_housing,
        X_test_housing_np, y_test_housing_np, 
        scaler_y=scaler_y_housing, # Pasar el scaler_y para desescalar en la evaluación si se usó
        device=device
    )
else:
    print("Modelo de regresión no entrenado o componentes no disponibles. Saltando evaluación.")

## 4. Conclusiones del Ejercicio (Regresión California Housing con PyTorch)

**Resumen de Hallazgos:**
* Se cargó y preprocesó el dataset California Housing, escalando las características de entrada (y opcionalmente el objetivo) y convirtiendo los datos a tensores de PyTorch. Se creó un conjunto de validación.
* Se definió una red neuronal (`HousingRegressorNet`) usando `torch.nn.Module` para la tarea de regresión.
* El modelo fue compilado con `MSELoss` y el optimizador `Adam` (con `weight_decay` para regularización L2).
* Se implementó un bucle de entrenamiento explícito con validación, `ReduceLROnPlateau`, `EarlyStopping` y `ModelCheckpointing`.
* El modelo alcanzó un Error Absoluto Medio (MAE) en el conjunto de prueba de **[Completar con MAE obtenido]** y un R² de **[Completar con R² obtenido]**. (El MAE debe interpretarse en la escala original del precio si el target fue desescalado).
* Las curvas de aprendizaje y los gráficos de residuales ayudaron a evaluar el ajuste del modelo.

**Aprendizaje General:**
Este ejercicio demostró la aplicación de PyTorch para un problema de regresión con redes neuronales. Se cubrió el preprocesamiento de datos para PyTorch, la definición de un modelo personalizado, la escritura de un bucle de entrenamiento detallado con control sobre optimizadores y schedulers, y la evaluación del modelo con métricas y visualizaciones relevantes. La flexibilidad de PyTorch permite un control granular sobre todo el proceso de modelado.

*(Nota: Los resultados específicos deben completarse después de ejecutar completamente el notebook.)*